**Pipeline Batch Bovespa (entrega obrigatória):**

- Requisito 1 - Scrap de dados do site da B3 com dados do pregão D-1;
- Requisito 2 - Os dados brutos devem ser ingeridos no s3 em formato parquet com partição diária; 
- Requisito 3 - O bucket deve acionar uma lambda que por sua vez irá chamar o job de ETL no glue; 
- Requisito 4 - A lambda pode ser em qualquer linguagem. Ela apenas deverá iniciar o job Glue; 
- Requisito 5 - O Job Glue deve ser feito no modo visual. Este job deve conter as seguintes transformações obrigatórias:
    - A - Agrupamento numérico, sumarização, contagem ou soma.
    - B - Renomear duas colunas existentes além das de agrupamento.
    - C - Realizar um cálculo com campos de data, Exemplo, poder ser duração, comparação, diferença entre datas. 
- Requisito 6 - Os dados refinados no job glue devem ser salvos no formato parquet, numa pasta chamada refined, particionado por data e pelo nome ou abreviação da ação do pregão.
- Requisito 7 - O job Glue deve automaticamente catalogar o dado no Glue Catalog e criar uma tabela no banco de dados default do Glue Catalog. 
- Requisito 8 - Os dados devem estar disponíveis, legíveis no Athena. 
- Requisito 9 - Opcional construir um notebook no Athena para montar uma visualização gráfica dos dados ingeridos. Pipeline Stream Bitcoin (entrega opcional):


![image.png](attachment:3bb2fc51-40a3-4465-a5f3-2afefc234013.png)

In [6]:
# !pip install selenium
# !pip install webdriver-manager
# !pip install pandas
# !pip install pyarrow
# !pip install boto3

In [6]:
import os
import time
import boto3
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

def download_file(URL):

    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(URL)
    time.sleep(3)
    
    navegador.find_element('xpath', '//*[@id="segment"]').send_keys('Setor de Atuação')
    time.sleep(3)
    
    navegador.find_element('xpath', '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a').click()
    time.sleep(3)
    return True

def get_file_path(path, prefix):
    files = os.listdir(path)
    files = [k for k in files if prefix in k]
    file = sorted(files,reverse=True)[0]
    return path+file

def get_file_path(path, prefix):
    files = os.listdir(path)
    files = [k for k in files if prefix in k]
    file = sorted(files,reverse=True)[0]
    return path+file

def get_data_bovespa(file_path):
    date_pregao = pd.read_csv(file_path, sep=";", encoding='ISO-8859-1',nrows=1,index_col=False).columns[0].split(' ')[-1]
    data_bovespa = pd.read_csv(file_path, sep=";", encoding='ISO-8859-1',skipfooter=2, engine='python',thousands='.', decimal=',', header=1, index_col=False)
    
    data_bovespa['data_pregao'] = pd.to_datetime(date_pregao)
    data_bovespa['data_pregao'] = data_bovespa['data_pregao'].astype(str)
    data_bovespa = data_bovespa.rename(columns={'Setor': 'setor','Código': 'codigo','Ação': 'acao','Tipo': 'tipo','Qtde. Teórica': 'qtde_teorica','Part. (%)': 'part','Part. (%)Acum.': 'part_acum'})
    return data_bovespa

def main():
    URL = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/ibov?language=pt-br"
    PATH = '/home/aleksandro/Downloads/'

    print(f'>>Scrap de dados do site da B3 - URL{URL} ')
    download_file(URL)

    print('>>>Obtém caminho do arquivo: ')
    file_path = get_file_path(PATH, prefix='IBOV')

    print('>>>>Transformação inicial (capturar data e re-nomeação dos campos)')
    data_bovespa = get_data_bovespa(file_path)

    print('>>>>>Salva aruivo em parquet: ', PATH + 'bovespa.parquet')
    data_bovespa.head(3)
    data_bovespa.to_parquet(PATH + 'bovespa.parquet')

    print('>>>>>>Envia arquivo bovespa.parquet para zona RAW do bucket: ', PATH + 'bovespa.parquet')
    s3 = boto3.client('s3')
    s3.upload_file(PATH + '/bovespa.parquet', 'fiap-mlet-aleksandro', 'raw/bovespa.parquet')

    print('>>>>>>>Start do job de ETL no glue: ', PATH + 'bovespa.parquet')
    glue = boto3.client('glue')
    glue.start_job_run(JobName="tc-dados-bovespa")

if __name__ == "__main__":
    main()


>>Scrap de dados do site da B3 - URLhttps://sistemaswebb3-listados.b3.com.br/indexPage/day/ibov?language=pt-br 
>>>Obtém caminho do arquivo: 
>>>>Transformação inicial (capturar data e re-nomeação dos campos)
>>>>>Salva aruivo em parquet:  /home/aleksandro/Downloads/bovespa.parquet
>>>>>>Envia arquivo bovespa.parquet para zona RAW do bucket:  /home/aleksandro/Downloads/bovespa.parquet


/tmp/ipykernel_146436/3015201674.py:38: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  date_pregao = pd.read_csv(file_path, sep=";", encoding='ISO-8859-1',nrows=1,index_col=False).columns[0].split(' ')[-1]


>>>>>>>Start do job de ETL no glue:  /home/aleksandro/Downloads/bovespa.parquet


In [4]:
glue = boto3.client('glue')
glue.start_job_run(JobName="tc-dados-bovespa")

{'JobRunId': 'jr_6f6fbce942f4c52bee4655c9df8984d2105e260ac7f5c4ad9e8a96be4d4725e1',
 'ResponseMetadata': {'RequestId': '26978baf-2d6a-46a3-ac9a-0c94865a394f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 22 Jul 2024 13:17:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'connection': 'keep-alive',
   'x-amzn-requestid': '26978baf-2d6a-46a3-ac9a-0c94865a394f'},
  'RetryAttempts': 0}}